## quadtree & Barnes-Hut

https://www.cs.princeton.edu/courses/archive/fall03/cs126/assignments/barnes-hut.html  
http://www-inf.telecom-sudparis.eu/COURS/CSC5001/new_site/Supports/Projet/NBody/barnes_86.pdf  
https://en.wikipedia.org/wiki/Barnes%E2%80%93Hut_simulation

https://github.com/rdeits/RegionTrees.jl

https://github.com/JuliaArrays/StaticArrays.jl

In [1]:
using BenchmarkTools

In [2]:
mutable struct Cell
    center ::NTuple{2, Float64}
    size ::NTuple{2, Float64}
    nbr_of_points ::Int
    sum_of_points ::NTuple{2, Float64}
    childrens ::Array{Union{Nothing, Cell}, 1}
end;

Cell(center, size) = Cell(
    center,
    size,
    0,
    (0.0, 0.0),
    Array{Union{Nothing, Cell}}(nothing, (4, ))
);

In [3]:
Cell((0.5, 0.5), (1.0, 1.0))

Cell((0.5, 0.5), (1.0, 1.0), 0, (0.0, 0.0), Union{Nothing, Cell}[nothing, nothing, nothing, nothing])


1. If node x does not contain a body, put the new body b here.
2. If node x is an internal node, update the center-of-mass and total mass of x. Recursively insert the body b in the appropriate quadrant.
3. If node x is an external node, say containing a body named c, then there are two bodies b and c in the same region. Subdivide the region further by creating four children. Then, recursively insert both b and c into the appropriate quadrant(s). Since b and c may still end up in the same quadrant, there may be several subdivisions during a single insertion. Finally, update the center-of-mass and total mass of x. 

In [4]:
function which_quadrant(point, cell_center)
    x, y = point
    center_x, center_y = cell_center
    quadrant = 1
    if x > center_x
        quadrant += 1
        end;
    if y < center_y
        quadrant += 2
        end;
    return quadrant
    end;
            
# Numbering:
#   1 | 2
#  -------
#   3 | 4
# cell corner (x, y) is bottom left

In [5]:
const min_cell_size = 0.1

const offset = [(-0.25, 0.25),
                (0.25, 0.25),
                (-0.25, -0.25),
                (0.25, -0.25)]

function create_subcell(cell, quadrant)
    size = cell.size ./ 2 
    center = cell.center .+ (offset[quadrant].*cell.size)
    return Cell(center, size)
    end;

function get_subcell!(cell, point)
    quadrant = which_quadrant(point, cell.center)
    if isnothing(cell.childrens[quadrant])
        cell.childrens[quadrant] = create_subcell(cell, quadrant)
        end;
    return cell.childrens[quadrant]
    end;

function insert!(cell, point)
    # check if node in cell... no, or to growth outward
    
    if cell.nbr_of_points > 0 #&& min(cell.size...) > min_cell_size
    
        if cell.nbr_of_points == 1
            # insert further the old point
            previous_point = cell.sum_of_points
            subcell = get_subcell!(cell, previous_point)
            insert!(subcell, previous_point)
            end;
        
        # continue insertion of the point
        subcell = get_subcell!(cell, point)
        insert!(subcell, point)
        end;
    
    # update cell data
    cell.sum_of_points = cell.sum_of_points .+ point
    cell.nbr_of_points += 1     
    end;

In [6]:
root = Cell((0.5, 0.5), (1.0, 1.0))

Cell((0.5, 0.5), (1.0, 1.0), 0, (0.0, 0.0), Union{Nothing, Cell}[nothing, nothing, nothing, nothing])

In [7]:
function build_the_tree(points)
    x_min, x_max = extrema(getindex.(points, 1))
    y_min, y_max = extrema(getindex.(points, 2))
    center = ((x_max+x_min)/2, (y_max+y_min)/2)
    size = (x_max-x_min, y_max-y_min)
    root = Cell(center, size)
    foreach(xy -> insert!(root, xy), points)
    return root
    end;

In [8]:
distance(a, b) = sqrt(sum( (a .- b).^2 ));

In [9]:
# evaluate cell, x, y, theta=.5
# Returns list of points (effective) with their mass
function evaluate!(output, cell, point, theta=0.5)
    if isnothing(cell)
        return
        end;
    if cell.nbr_of_points == 1
        push!(output, (cell.sum_of_points, cell.nbr_of_points))
    else
        center_of_mass = cell.sum_of_points ./ cell.nbr_of_points
        angle = 0.5*sum(cell.size)/distance(point, center_of_mass)
    
        if angle < theta
            push!(output, (center_of_mass, cell.nbr_of_points))
        else
            foreach(quad -> evaluate!(output, quad, point, theta), cell.childrens)
            end;
        end;
    end;

In [10]:
random_points(N) = [Tuple(rand(2)) for k in 1:N];

In [11]:
@benchmark build_the_tree(points) setup=(points = random_points(1000))
# N=10       median time:      1.681 μs (0.00% GC)
# N=100      median time:      20.411 μs (0.00% GC)
# N=1000     median time:      229.715 μs (0.00% GC)
# N=10000    median time:      3.875 ms (0.00% GC)

BenchmarkTools.Trial: 
  memory estimate:  328.63 KiB
  allocs estimate:  3338
  --------------
  minimum time:     201.243 μs (0.00% GC)
  median time:      217.464 μs (0.00% GC)
  mean time:        267.348 μs (15.85% GC)
  maximum time:     3.762 ms (88.73% GC)
  --------------
  samples:          6273
  evals/sample:     1

In [12]:
points = random_points(100)
root = build_the_tree(points);

In [13]:
output = []
evaluate!(output, root, (.6, .7), 0.5);

In [14]:
@benchmark evaluate!(output, root, xy) setup=(xy=Tuple(rand(2)); output=[])
# N=10      median time:      517.299 ns (0.00% GC)
# N=100     median time:      2.431 μs (0.00% GC)
# N=1000    median time:      6.392 μs (0.00% GC)  
# N=10000   median time:      11.886 μs (0.00% GC)

BenchmarkTools.Trial: 
  memory estimate:  1.42 KiB
  allocs estimate:  28
  --------------
  minimum time:     1.271 μs (0.00% GC)
  median time:      2.621 μs (0.00% GC)
  mean time:        3.655 μs (27.27% GC)
  maximum time:     5.799 ms (99.87% GC)
  --------------
  samples:          10000
  evals/sample:     10

### brute force

In [15]:
@benchmark distance(u, v) setup=(u=Tuple(rand(2)); v=Tuple(rand(2)))
#   median time:      23.619 ns (0.00% GC)

BenchmarkTools.Trial: 
  memory estimate:  16 bytes
  allocs estimate:  1
  --------------
  minimum time:     22.813 ns (0.00% GC)
  median time:      23.628 ns (0.00% GC)
  mean time:        34.071 ns (23.26% GC)
  maximum time:     56.932 μs (99.92% GC)
  --------------
  samples:          10000
  evals/sample:     996

In [16]:
alldistances(points, u) = map(x -> distance(x, u), points)

alldistances (generic function with 1 method)

In [17]:
@benchmark alldistances(points, x) setup=(points=random_points(10000); x=Tuple(rand(2)))
# N=100     median time:      422.844 ns (0.00% GC)
# N=1000    median time:      3.980 μs (0.00% GC)
# N=10k     median time:      41.106 μs (0.00% GC)

BenchmarkTools.Trial: 
  memory estimate:  78.23 KiB
  allocs estimate:  3
  --------------
  minimum time:     37.358 μs (0.00% GC)
  median time:      40.344 μs (0.00% GC)
  mean time:        43.450 μs (4.49% GC)
  maximum time:     1.948 ms (97.51% GC)
  --------------
  samples:          974
  evals/sample:     1